# Philodoptera Dataset Conversion

Stridulation in crickets (e.g. [Pholidoptera littoralis](https://www.orthoptera.ch/wiki/arten/ensifera/tettigoniinae/item/pholidoptera-littoralis)) is a great animal model to study motor control/ sound production. Below is a tutorial on how we can create a `trials.nc`. Note that we have a video, audio and DeepLabCut pose file, meaning we can look at **movement** and **sound** features in the GUI!

<img src="assets/cricket1.png" width="900">
<img src="assets/cricket2.png" width="900">

Made in Slovenia, 2025.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import xarray as xr
from pathlib import Path
from audioio import load_audio

from movement.io import load_poses
from movement.kinematics import compute_velocity, compute_speed, compute_acceleration


from ethograph import set_media_attrs, minimal_basics, get_project_root
from ethograph.features.audio_features import get_envelope

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
project_root = Path.cwd().parent
os.chdir(project_root)

data_folder = get_project_root() / "data" / "examples"
poses_csv_path = data_folder / "philodoptera.csv"
audio_path =  data_folder/ "philodoptera.wav"
video_path = data_folder / "philodoptera.mp4"

fps = 240

In [3]:
# The "philodoptera.csv" contains position (xy) data of the left/right wing tip
ds = load_poses.from_dlc_file(poses_csv_path, fps=fps)

# Derive some kinematic features for wing position
ds["velocity"] = compute_velocity(ds.position)
ds["speed"] = compute_speed(ds.position)
ds["acceleration"] = compute_acceleration(ds.position)

In [4]:
audio, sr = load_audio(str(audio_path))
ds.attrs["audio_sr"] = sr

ds['individuals'] = ["Pholidoptera_littoralis_1"]
ds.attrs["Recording info"] = "Akseli Ilmanen: This cricket species Pholidoptera littoralis was recorded on a field trip in Slovenia in 2025. The video was recorded with an iPhone and the audio with an Zoom recorder. Synchronization may not be perfect."

In [5]:
# Get envelope from audio data at custom sampling rate
env_rate = 500
envelope, _ = get_envelope(audio_path, sr, env_rate)

time_env = np.arange(len(envelope)) / env_rate
ds['envelope'] = xr.DataArray(
    envelope,
    dims=['time_env', "channels"],
    coords={'time_env': time_env}
)

In [46]:
dt.save()

In [42]:
path = r"C:\Users\aksel\Documents\Code\EthoGraph\data\examples\copExpBP08_trim.nc"
from ethograph import TrialTree

dt = TrialTree.open(path)


In [41]:
print(dt)

<xarray.DataTree>
Group: /
└── Group: /trial_sample_trial
        Dimensions:                    (time: 122070, individuals: 2)
        Coordinates:
          * time                       (time) float64 977kB 20.0 20.0 20.0 ... 25.0 25.0
          * individuals                (individuals) <U21 168B 'male (red radio)' 'fe...
        Data variables:
            vibration (accelerometer)  (time, individuals) float32 977kB 6.235e+05 .....
            labels                     (time, individuals) float64 2MB 0.0 0.0 ... 0.0
        Attributes:
            fps:       47.6837158203125
            audio_sr:  24414.0625
            cameras:   ['copExpBP08_trim.mp4']
            mics:      ['copExpBP08_trim.wav']
            trial:     sample_trial


In [39]:
print(dt.ds.data_vars)

Data variables:
    *empty*


In [27]:
dt["trial_sample_trial"].drop_vars("labels")

AttributeError: 'DataTree' object has no attribute 'drop_vars'

In [24]:
del dt["trial_sample_trial"].labels

AttributeError: 'DataTree' object has no attribute 'labels'

In [16]:
dt.itrial(0).attrs["mics"] = ["copExpBP08_trim.wav"]

In [6]:
# Necessary I/0 steps
ds = set_media_attrs(
    ds,
    cameras=[video_path.name],
    mics=[audio_path.name],
    pose=[poses_csv_path.name],
)

dt = minimal_basics(ds)

output_path = data_folder / "philodoptera.nc"
dt.to_netcdf(output_path)

Extracted type_vars_dict: {'individuals': array(['Pholidoptera_littoralis_1'], dtype='<U25'), 'cameras': array(['philodoptera.mp4'], dtype='<U16'), 'mics': array(['philodoptera.wav'], dtype='<U16'), 'pose': array(['philodoptera.csv'], dtype='<U16'), 'features': ['position', 'velocity', 'speed', 'acceleration', 'envelope'], 'channels': array([0, 1]), 'keypoints': array(['LeftWingTip', 'RightWingTip'], dtype='<U12'), 'space': array(['x', 'y'], dtype='<U1'), 'trial_conditions': []}
